# CoexistAI Tool Tutorial

Welcome to the tutorial for the coexistAI tool! This notebook will guide you through the main functionalities of the tool, including web search, document processing, generative models, answer generation, YouTube summarization, and more. Each section contains explanations and code examples to help you get started quickly.

## 1. Setup and Initialization

First, let's import the required libraries, set up environment variables, and initialize the main components. This ensures that all dependencies are loaded and ready for use.

In [ ]:
from utils.utils import *
from utils.websearch_utils import *
set_logging(True) 
from langchain_text_splitters import TokenTextSplitter
import os

if not is_searxng_running():
       !docker run --rm \
                 -d -p 30:8080 \
                 -v "${PWD}/searxng:/etc/searxng" \
                 -e "BASE_URL=http://localhost:$PORT/" \
                 -e "INSTANCE_NAME=my-instance" \
                 searxng/searxng
else:
       print("SearxNG docker container is already running.")


os.environ['GOOGLE_API_KEY'] = 'YOUR_LLM_KEY'  # Replace with your actual if google models are being used

text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=128)
from utils.websearch_utils import *
searcher = SearchWeb(30)  # Initialize web search with a result limit

9196753c99e73b257a72e2db9071a7e1886ffbd7acbb9c33bf9da134ea10b784
docker: Error response from daemon: failed to set up container networking: driver failed programming external connectivity on endpoint cool_jackson (ae02964745dc0f3d5aa815a4f650f7c044ffaa9b427b4782b0c33478a561ff30): Bind for 0.0.0.0:30 failed: port is already allocated

Run 'docker run --help' for more information


## 2. Loading Models

Load embedding models and cross-encoders using the `load_model` function. You can choose between different embedding modes such as 'gemini', 'huggingface', or 'infinity_emb'.

for local embedders, either use infinity_emb  or huggingface

In [9]:
hf_embeddings, cross_encoder = load_model("models/embedding-001", _embed_mode='google',
                                          kwargs={'api_key': os.environ['GOOGLE_API_KEY']})

[2025-07-27 12:05:45,487] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: google
[2025-07-27 12:05:45,487] INFO utils.utils: Loading model: models/embedding-001 with embedding mode: google
[2025-07-27 12:05:45,749] INFO transformers.configuration_utils: loading configuration file config.json from cache at /Users/sidhantthole/.cache/huggingface/hub/models--BAAI--bge-reranker-base/snapshots/2cfc18c9415c912f9d8155881c133215df768a70/config.json
[2025-07-27 12:05:45,749] INFO transformers.configuration_utils: loading configuration file config.json from cache at /Users/sidhantthole/.cache/huggingface/hub/models--BAAI--bge-reranker-base/snapshots/2cfc18c9415c912f9d8155881c133215df768a70/config.json
[2025-07-27 12:05:45,751] INFO transformers.configuration_utils: Model config XLMRobertaConfig {
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id":

## 4. Web Search Integration

Use the `SearchWeb` class to perform web searches and retrieve results. This is useful for augmenting LLMs with up-to-date information from the web.

In [10]:
# Get the top 3 results from querying the web
results = searcher.query_search("latest news in AI", num_results=3)
print(results)

[2025-07-27 12:05:51,924] INFO utils.websearch_utils: Search results for query 'latest news in AI': [{'snippet': 'SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to audit models for safety · Sam Altman: AI will cause job losses and ...', 'title': 'AI News | Latest AI News, Analysis & Events', 'link': 'https://www.artificialintelligence-news.com/', 'engines': ['google'], 'category': 'general'}, {'snippet': 'AI · Meta names Shengjia Zhao as chief scientist of AI superintelligence unit · AI referrals to top websites were up 357% year-over-year in June, reaching 1.13B.', 'title': 'AI News & Artificial Intelligence', 'link': 'https://techcrunch.com/category/artificial-intelligence/', 'engines': ['google'], 'category': 'general'}, {'snippet': "2 Jul 2025 — Here's a recap of some of our biggest AI updates from June, including more ways to search with AI Mode, a new way to share your NotebookLM notebooks publicly.", 'title': 'The latest AI news we ann

[
    {
        'snippet': 'SoftBank chief: Forget AGI, ASI will be here within 10 years · Anthropic deploys AI agents to 
audit models for safety · Sam Altman: AI will cause job losses and ...',
        'title': 'AI News | Latest AI News, Analysis & Events',
        'link': 'https://www.artificialintelligence-news.com/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': 'AI · Meta names Shengjia Zhao as chief scientist of AI superintelligence unit · AI referrals to
top websites were up 357% year-over-year in June, reaching 1.13B.',
        'title': 'AI News & Artificial Intelligence',
        'link': 'https://techcrunch.com/category/artificial-intelligence/',
        'engines': ['google'],
        'category': 'general'
    },
    {
        'snippet': "2 Jul 2025 — Here's a recap of some of our biggest AI updates from June, including more ways to
search with AI Mode, a new way to share your NotebookLM notebooks publicly.",
        'title': 'The latest AI news we announced in June',
        'link': 'https://blog.google/technology/ai/google-ai-updates-june-2025/',
        'engines': ['google'],
        'category': 'general'
    }
]

## 5. Document Conversion from URLs

Convert URLs into document objects using the `urls_to_docs` function. This allows you to process and analyze web content as structured documents.

In [11]:
# Scrape content from URLs and convert to documents
docs = await urls_to_docs([
    "https://en.wikipedia.org/wiki/India",
    "https://en.wikipedia.org/wiki/Bangalore"
])
print(f"Loaded {len(docs)} documents.")
print(docs[0])

/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-07-27 12:05:53,392] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/India
[2025-07-27 12:05:53,392] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/India
[2025-07-27 12:05:53,395] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/Bangalore
[2025-07-27 12:05:53,395] INFO utils.websearch_utils: Fetching URL: https://en.wikipedia.org/wiki/Bangalore
[2025-07-27 12:05:53,696] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/Bangalore with type text/html; charset=UTF-8
[2025-07-27 12:05:53,696] INFO utils.websearch_utils: Fetched content from https://en.wikipedia.org/wiki/Bangalore with type 

Loaded 2 documents.

Document(
    metadata={'source': 'https://en.wikipedia.org/wiki/India'},
    page_content='India - Wikipedia\n\nJump to content\n\nCoordinates: 21Â°N 78Â°E\ufeff / \ufeff21Â°N 78Â°E\ufeff 
/ 21; 78\n\n![Featured \n\n![Extended-protected \n\nFrom Wikipedia, the free encyclopedia\n\nCountry in South 
Asia\n\nThis article is about the country. For other uses, see India (disambiguation).\n\n| Republic of India  
*BhÄ\x81rat Gaá¹\x87arÄ\x81jya* | |\n| --- | --- |\n| Horizontal tricolour flag bearing, from top to bottom, deep 
saffron, white, and green horizontal bands. In the centre of the white band is a navy-blue wheel with 24 spokes.  
Flag  State emblem | |\n| **Motto:**Satyameva Jayate\xa0(Sanskrit) "Truth Alone Triumphs"[1] | |\n| 
**Anthem:**\xa0Jana Gana Mana\xa0(Hindi)[a][2][3] "Thou Art the Ruler of the Minds of All People"[4][2] | |\n| 
**National song: Vande Mataram\xa0(Sanskrit)[c]** "I Bow to Thee, Mother"[b][1][2] | |\n| Image of a globe centred 
on India, with India highlighted. Territory controlled by India  Territory claimed but not controlled | |\n| 
Capital | New Delhi 28Â°36â\x80²50â\x80³N 77Â°12â\x80²30â\x80³Eï»¿ / ï»¿28.61389Â°N 77.20833Â°Eï»¿ / 28.61389; 
77.20833 |\n| Largest city by city proper population | Mumbai |\n| Largest city by metropolitan area population | 
Delhi |\n| Official\xa0languages |  |\n| Recognised regional\xa0languages | State level and Eighth Schedule[9] |\n|
Native languages | 424 languages[g] |\n| Religion (2011)[11] |  |\n| Demonym(s) |  |\n| Government | Federal 
parliamentary republic |\n|  | |\n| â\x80¢\xa0President | Droupadi Murmu |\n| â\x80¢\xa0Prime Minister | Narendra 
Modi |\n|  | |\n| Legislature | Parliament |\n| â\x80¢\xa0Upper house | Rajya Sabha |\n| â\x80¢\xa0Lower house | 
Lok Sabha |\n| Independence from the United Kingdom | |\n|  | |\n| â\x80¢\xa0Dominion | 15 August 1947 |\n| 
â\x80¢\xa0Republic | 26 January 1950 |\n|  | |\n| Area | |\n| â\x80¢\xa0Total | 3,287,263\xa0km2 
(1,269,219\xa0sq\xa0mi)[2][h] (7th) |\n| â\x80¢\xa0Water\xa0(%) | 9.6 |\n| Population | |\n| â\x80¢\xa02023 
estimate | Neutral increase 1,428,627,663[13] (1st) |\n| â\x80¢\xa02011\xa0census | Neutral increase  (2nd) |\n| 
â\x80¢\xa0Density | 430.5/km2 (1,115.0/sq\xa0mi) (30th) |\n| GDP\xa0(PPP) | 2025\xa0estimate |\n| â\x80¢\xa0Total |
Increase $17.647 trillion[16] (3rd) |\n| â\x80¢\xa0Per capita | Increase $12,132[16] (119th) |\n| GDP\xa0(nominal) 
| 2025\xa0estimate |\n| â\x80¢\xa0Total | Increase $4.187 trillion[16] (4th) |\n| â\x80¢\xa0Per capita | Increase 
$2,878[16] (136th) |\n| Gini\xa0(2021) | Positive decrease\xa025.5[17] low inequality |\n| HDI\xa0(2023) | 
Increase\xa00.685[18] medium\xa0(130th) |\n| Currency | Indian rupee (â\x82¹) (INR) |\n| Time zone | UTC+05:30 
(IST) |\n| Date format |  |\n| Calling code | +91 |\n| ISO 3166 code | IN |\n| Internet TLD | .in (others) 
|\n\n**India**, officially the **Republic of India**,[j][20] is a country in South Asia. It is the seventh-largest 
country by area; the most populous country since 2023;[21] and, since its independence in 1947, the world\'s most 
populous  Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the 
southeast, it shares land borders with Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and 
Bangladesh and Myanmar to the east. In the Indian Ocean, India is near Sri Lanka and the Maldives; its Andaman and 
Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.\n\nModern humans arrived on the 
Indian subcontinent from Africa no later than 55,000 years ago.[26][27][28] Their long occupation, predominantly in
isolation as hunter-gatherers, has made the region highly diverse.[29] Settled life emerged on the subcontinent in 
the western margins of the Indus river basin 9,000 years ago, evolving gradually into the Indus Valley Civilisation
of the third millennium BCE.[30] By 1200\xa0BCE, an archaic form of Sanskrit, an Indo-European langu

## 6. Using Generative Models

Initialize and use generative models like `ChatGoogleGenerativeAI` and `ChatOpenAI` for text generation tasks. The `get_generative_model` function helps you select and configure the model.


### For local llms

```python
llmlocal = get_generative_model(
    model_name="google/gemma-3-12b",
    type='local',
    base_url="http://127.0.0.1:1234/v1", # for an example if you are running lmstudio locally
    _tools=None,
    kwargs={
        "temperature": 0.1,  # Sampling temperature for generation.
        "max_tokens": None,  # Maximum number of tokens to generate (None for default).
        "timeout": None,     # Timeout for API requests (None for default).
        "max_retries": 2,    # Maximum number of retries for failed requests.
        "api_key": "dummy",  # API key for authentication.
    }
)
```

You can even load local models from ollama etc, just configure openai compatible API server base url arg
For other proprietary models as well, just configure the base_url arg accordingly with "others" as type


In [12]:
llmgoogle = get_generative_model(model_name='gemini-1.5-flash',
                    type='google',
                    _tools=None,
                    kwargs={'temperature': 0.1, 'max_tokens': None, 'timeout': None, 'max_retries': 2, 
                            'api_key': os.environ['GOOGLE_API_KEY'],
                            'generation_config':{"response_mime_type": "application/json"}})

[2025-07-27 12:05:59,337] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
[2025-07-27 12:05:59,337] WARNING langchain_google_genai.chat_models: Unexpected argument 'generation_config' provided to ChatGoogleGenerativeAI.
/var/folders/sh/v0nx5vbx4mv07k1h4mxmwxcw0000gn/T/ipykernel_53771/4124600085.py:1: UserWarning: WARNING! generation_config is not default parameter.
                generation_config was transferred to model_kwargs.
                Please confirm that generation_config is what you intended.
  llmgoogle = get_generative_model(model_name='gemini-1.5-flash',


# 7. Query -> Search -> Scrape -> Response 
```query_web_response```, intelligently choses between to RAG and Summary of full page based on the request

In [13]:
web_response = await query_web_response(
    "Top news of today in science and technology with date and link to original article as hyperlink and summary of each in one line",
    '26-07-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=[],
    local_mode=False,
    split=False
)
print(web_response[0])

[2025-07-27 12:06:02,440] INFO utils.answer_generation: Summary:True,subquery:['Top science news today', 'Top technology news today', 'Links to news articles', 'Summary of science news', 'Summary of technology news', 'Date of news articles'], iscoverd:False
[2025-07-27 12:06:02,440] INFO utils.answer_generation: Summary:True,subquery:['Top science news today', 'Top technology news today', 'Links to news articles', 'Summary of science news', 'Summary of technology news', 'Date of news articles'], iscoverd:False
[2025-07-27 12:06:02,442] INFO utils.websearch_utils: Search phrases for query 'Top news of today in science and technology with date and link to original article as hyperlink and summary of each in one line': ['top science news today', 'top technology news today', 'links to news articles', 'summary of science news', 'summary of technology news', 'date of news articles']
[2025-07-27 12:06:02,442] INFO utils.websearch_utils: Search phrases for query 'Top news of today in science a

Summarising using 3 documents

[2025-07-27 12:06:19,947] INFO utils.websearch_utils: Response generated for query 'Top news of today in science and technology with date and link to original article as hyperlink and summary of each in one line'.
[2025-07-27 12:06:19,947] INFO utils.websearch_utils: Response generated for query 'Top news of today in science and technology with date and link to original article as hyperlink and summary of each in one line'.


Top Science and Technology News of Today (July 27, 2025):

**Note:**  The provided text does not contain specific dates for most articles, so I'm assuming today's date, July 
27th, 2025, for all entries.  Links are not directly provided in the text, only source websites.  I will provide 
summaries in one line where possible.


**From ScienceNews.org:**

1.  **Maggots may have been on the Neandertal menu:**  A study suggests Neandertals may have eaten maggots for 
nutritional benefits. [Link:  https://www.sciencenews.org/](This link is only the website, not a direct article 
link.)
2.  **This desert beetle runs to cool off:**  A desert beetle uses running to regulate its body temperature. [Link:
https://www.sciencenews.org/](This link is only the website, not a direct article link.)
3.  **AI is designing proteins that could help treat cancer:** Artificial intelligence is being used to develop 
cancer-fighting proteins. [Link: https://www.sciencenews.org/](This link is only the website, not a direct article 
link.)
4.  **An injected gel could make drugs like Ozempic last longer:** A new gel could extend the effectiveness of 
medications like Ozempic. [Link: https://www.sciencenews.org/](This link is only the website, not a direct article 
link.)
5.  **How many steps a day do you really need to take?:**  A discussion on the optimal number of daily steps for 
health benefits. [Link: https://www.sciencenews.org/](This link is only the website, not a direct article link.)
6.  **Climate change may be pushing fungal allergy season earlier:**  Climate change is potentially affecting the 
timing of fungal allergy seasons. [Link: https://www.sciencenews.org/](This link is only the website, not a direct 
article link.)
7.  **AI reveals new details about a famous Latin inscription:** Artificial intelligence helps uncover new 
information from an ancient Latin inscription. [Link: https://www.sciencenews.org/](This link is only the website, 
not a direct article link.)


**From IndianExpress.com (Technology Section):**

1.  **Amazfit Active 2 (Square) review:** A review of the Amazfit Active 2 Square smartwatch. [Link:  
https://indianexpress.com/section/technology/](This link is only the website, not a direct article link.)
2.  **Gemini on Galaxy Watch review:**  A user experience review of using the Gemini AI assistant on a Galaxy 
Watch. [Link: https://indianexpress.com/section/technology/](This link is only the website, not a direct article 
link.)
3.  **SpaceX launches more Starlink satellites:** SpaceX added 52 more Starlink satellites to its constellation. 
[Link: https://indianexpress.com/section/technology/](This link is only the website, not a direct article link.)
4.  **New theory predicts a Big Crunch:** A new cosmological theory suggests a potential "Big Crunch" in the 
distant future. [Link: https://indianexpress.com/section/technology/](This link is only the website, not a direct 
article link.)


**Assumptions Made:**

*   All articles from both sources are assumed to be from July 27, 2025, as the provided text lacks specific dates 
for most articles.
*   The links provided are to the source websites, not direct links to individual articles.  Finding the exact 
articles would require navigating the websites.

# 8. Summarise/QA any web page
You can even summarise any url, and get the response tailored to your specific query

In [14]:
summary = await summary_of_url("Plan my 5 day trip to andaman and nicobar based on this",
               "https://www.twofortheworld.com/andaman-itinerary/",
               llmgoogle)
print(summary)

/opt/homebrew/Cellar/python@3.13/3.13.1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/multiprocessing/resource_tracker.py:136: UserWarning: resource_tracker: process died unexpectedly, relaunching.  Some resources might leak.
  warnings.warn('resource_tracker: process died unexpectedly, '
[2025-07-27 12:06:29,297] INFO utils.websearch_utils: Fetching URL: https://www.twofortheworld.com/andaman-itinerary/
[2025-07-27 12:06:29,297] INFO utils.websearch_utils: Fetching URL: https://www.twofortheworld.com/andaman-itinerary/
[2025-07-27 12:06:31,514] INFO utils.websearch_utils: Fetched content from https://www.twofortheworld.com/andaman-itinerary/ with type text/html; charset=UTF-8
[2025-07-27 12:06:31,514] INFO utils.websearch_utils: Fetched content from https://www.twofortheworld.com/andaman-itinerary/ with type text/html; charset=UTF-8
[2025-07-27 12:06:33,154] INFO utils.websearch_utils: Processed markdown for: https://www.twofortheworld.com/andaman-itinerary/
[2025-07-27 12:

This 10-day Andaman and Nicobar Islands itinerary focuses on three islands: South Andaman (Port Blair), Havelock, 
and Neil.  A 5-day trip would require significant adjustments, prioritizing fewer locations.

**Possible 5-Day Itinerary (requiring flexibility):**

* **Days 1-2: Port Blair:** Explore Cellular Jail, Ross Island, and the Zonal Anthropology Museum. Consider a 
shorter trip to Chidiya Tapu Beach if time allows.  Skip Baratang Limestone Caves due to ethical concerns regarding
the Jarawa tribe.
* **Days 3-4: Havelock Island:** Focus on either Radhanagar Beach (famous sunset) or Elephant Beach (jungle trek, 
tide dependent).  Scuba diving is possible but may need pre-booking and depends on weather.
* **Day 5: Neil Island:** Choose one beach (Laxmanpur II for sunset and rock pools or Sitapur for sunrise and a 
relaxed atmosphere).  Return to Port Blair for your departure.

**Important Considerations:**

* **Ferries:** Ferry travel between islands is crucial and can be unpredictable, especially during monsoon season 
(May-September).  Book private ferries (Makruzz or Green Ocean) in advance for more reliability, though they are 
more expensive than government ferries.
* **Weather:** October to May is the best time to visit. Be prepared for potential delays and adjust your itinerary
accordingly.
* **Time Constraints:** A 5-day trip necessitates prioritizing locations.  Choosing between Havelock and Neil might
be necessary for a more relaxed experience.


This condensed itinerary sacrifices some of the exploration offered in the original 10-day plan but provides a 
framework for a shorter, impactful Andaman experience.  Remember to check ferry schedules and weather forecasts 
before and during your trip.

 
# Summarise or QA any folder, file (including images/pdfs/excels/csv/pptx/etc), or code-file

In [15]:
#FOLDER
web_response = await query_web_response(
    "Whats all in DOCUMENT folder, enlist name with content in detail",
    '26-07-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=['documents'],
    local_mode=True,
    split=False
)
print(web_response[0])

[2025-07-27 12:06:58,344] INFO utils.answer_generation: Summary:True,subquery:['List files in DOCUMENT folder', 'Describe content of each file in DOCUMENT folder'], iscoverd:False
[2025-07-27 12:06:58,344] INFO utils.answer_generation: Summary:True,subquery:['List files in DOCUMENT folder', 'Describe content of each file in DOCUMENT folder'], iscoverd:False
[2025-07-27 12:06:58,346] INFO utils.websearch_utils: Search phrases for query 'Whats all in DOCUMENT folder, enlist name with content in detail': ['list files in document folder', 'describe content of each file in document folder']
[2025-07-27 12:06:58,346] INFO utils.websearch_utils: Search phrases for query 'Whats all in DOCUMENT folder, enlist name with content in detail': ['list files in document folder', 'describe content of each file in document folder']
[2025-07-27 12:06:58,347] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:06:58,347] WARNING utils.websear

Summarising using 2 documents

[2025-07-27 12:07:12,104] INFO utils.websearch_utils: Response generated for query 'Whats all in DOCUMENT folder, enlist name with content in detail'.
[2025-07-27 12:07:12,104] INFO utils.websearch_utils: Response generated for query 'Whats all in DOCUMENT folder, enlist name with content in detail'.


The DOCUMENT folder contains one file: `documents/1706.03762v7.pdf`.  This PDF is a research paper titled 
"Attention is All You Need," detailing the Transformer model architecture.

The paper's content includes:

* **Authors and Affiliations:**  A list of authors and their affiliations (Google Research and Google Brain 
primarily, with one author from the University of Toronto).  Their individual contributions are also specified.

* **Abstract:** A summary of the paper, highlighting the introduction of the Transformer model, its superior 
performance and parallelization capabilities compared to recurrent and convolutional neural networks in machine 
translation tasks, and its successful application to English constituency parsing.

* **Introduction:** Background on sequence transduction models, particularly recurrent neural networks (RNNs) and 
their limitations regarding parallelization.  It introduces the Transformer as a solution that relies solely on 
attention mechanisms.

* **Background:**  A review of related work, including convolutional neural networks (CNNs) and self-attention 
mechanisms.

* **Model Architecture:** A detailed explanation of the Transformer architecture, including:
    * Encoder and Decoder Stacks:  The structure of the encoder and decoder, composed of stacked layers with 
self-attention and feed-forward networks.
    * Attention:  A description of the scaled dot-product attention and multi-head attention mechanisms.
    * Position-wise Feed-Forward Networks:  Explanation of the fully connected feed-forward networks applied to 
each position.
    * Embeddings and Softmax:  Details on token embedding and probability prediction.
    * Positional Encoding:  How positional information is incorporated into the model.

* **Why Self-Attention:** A comparison of self-attention with recurrent and convolutional layers, focusing on 
computational complexity, parallelization, and path length for long-range dependencies.

* **Training:**  Details on the training data (WMT 2014 English-German and English-French datasets), hardware (8 
NVIDIA P100 GPUs), optimizer (Adam), and regularization techniques (residual dropout and label smoothing).

* **Results:**  Presentation of results on machine translation tasks (English-to-German and English-to-French), 
showing state-of-the-art BLEU scores achieved by the Transformer model.  Results on English constituency parsing 
are also included, demonstrating the model's generalizability.

* **Model Variations:**  An analysis of different variations of the Transformer architecture and their impact on 
performance.

* **Conclusion:**  Summary of the findings and future research directions.

* **References:** A list of cited works.

* **Appendix (partially shown):** Attention visualizations illustrating the model's attention mechanisms.


**Assumptions Made:**

No assumptions were made beyond what is explicitly stated or implied within the provided document.  The summary is 
purely based on the content of the research paper.

In [16]:
# FILE
web_response = await query_web_response(
    "Explain me the capabilties of this app in markdown table format",
    '26-07-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=['app.py'],
    local_mode=True,
    split=False
)
print(web_response[0])

[2025-07-27 12:07:20,155] INFO utils.answer_generation: Summary:True,subquery:['App capabilities markdown table'], iscoverd:False
[2025-07-27 12:07:20,155] INFO utils.answer_generation: Summary:True,subquery:['App capabilities markdown table'], iscoverd:False
[2025-07-27 12:07:20,157] INFO utils.websearch_utils: Search phrases for query 'Explain me the capabilties of this app in markdown table format': ['app capabilities markdown table']
[2025-07-27 12:07:20,157] INFO utils.websearch_utils: Search phrases for query 'Explain me the capabilties of this app in markdown table format': ['app capabilities markdown table']
[2025-07-27 12:07:20,158] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:07:20,158] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:07:20,159] INFO utils.websearch_utils: Total paths for 'Explain me the capabilties of this app in markdown 

Summarising using 4 documents

[2025-07-27 12:07:33,982] INFO utils.websearch_utils: Response generated for query 'Explain me the capabilties of this app in markdown table format'.
[2025-07-27 12:07:33,982] INFO utils.websearch_utils: Response generated for query 'Explain me the capabilties of this app in markdown table format'.


## CoexistAI App Capabilities

The CoexistAI application offers a range of functionalities, all powered by a large language model (LLM) and 
various supporting tools.  The following table summarizes its capabilities:

| Feature             | Description                                                                                
| Input Parameters                                                                                                 
| Output                                                                                                    | 
Assumptions                                                                                                    |
|----------------------|-------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------
-------------------------|-----------------------------------------------------------------------------------------
--------------------------------|----------------------------------------------------------------------------------
---------------------------|---------------------------------------------------------------------------------------
----------------------|
| Web Search           | Performs a web search using SearxNG, retrieves results, and generates a response based on 
those results.  Includes options for reranking and local document processing.                                      
| `query`, `rerank`, `num_results`, `local_mode`, `split`, `document_paths`                                        
| Generated response, sources, search results, retrieved documents, and context.                               | 
SearxNG docker container is either running or can be started automatically.                               |
| Web Summarization    | Generates a summary of a web page based on a provided query and URL.                      
| `query`, `url`, `local_mode`                                                                                     
| Summary and sources                                                                                           |  
A suitable summarization model is available and configured.                                                  |
| YouTube Search       | Performs a YouTube search, retrieves transcripts, and generates summaries of the videos 
based on a given query or URL.                                                                                     
| `query` (URL or search term), `prompt`, `n` (number of videos)                                                   
| Summaries of YouTube transcripts                                                                               | 
YouTube transcripts are accessible and can be processed.                                                    |
| Reddit Search        | Performs a Reddit search and retrieves posts based on specified parameters (subreddit, 
search query, time filter, etc.).                                                                                  
| `subreddit`, `url_type`, `n`, `k`, `custom_url`, `time_filter`, `search_query`, `sort_type`                      
| Reddit posts and comments                                                                                      | 
Reddit API access is configured correctly.                                                                   |
| Map Search           | Performs a map search to find routes and points of interest (POIs) between two locations. 
| `start_location`, `end_location`, `pois_radius`, `amenities`, `limit`, `task`                                    
| Location(s), route, and POIs                                                                                  | 
Map data is accessible and can be processed.                                                                 |
| Git Repository Search | Retrieves the directory tree structure or specific content from a

In [17]:
# FILE
web_response = await query_web_response(
    "Give me your artistic view on this logo",
    '26-07-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=['artifacts/logo.jpeg'],
    local_mode=True,
    split=False
)
print(web_response[0])

[2025-07-27 12:07:47,909] INFO utils.answer_generation: Summary:False,subquery:['artistic view logo'], iscoverd:False
[2025-07-27 12:07:47,909] INFO utils.answer_generation: Summary:False,subquery:['artistic view logo'], iscoverd:False
[2025-07-27 12:07:47,911] INFO utils.websearch_utils: Search phrases for query 'Give me your artistic view on this logo': ['artistic view logo']
[2025-07-27 12:07:47,911] INFO utils.websearch_utils: Search phrases for query 'Give me your artistic view on this logo': ['artistic view logo']
[2025-07-27 12:07:47,912] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:07:47,912] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:07:47,913] INFO utils.websearch_utils: Total paths for 'Give me your artistic view on this logo': [['artifacts/logo.jpeg']]
[2025-07-27 12:07:47,913] INFO utils.websearch_utils: Total paths for 'Give me yo

#### Answer: 
The logo is a vibrant and imaginative illustration blending nature, science, and everyday life. A lion in a suit 
holding coffee represents power and sophistication, while a blue brain above symbolizes intellect. Two bulls, one 
blue and one black and white, represent contrasting perspectives.  The background features interconnected tech 
icons (gears, test tubes, cameras, circuits), suggesting innovation and the complexity of the mind. Stars add a 
whimsical touch. The overall artistic effect is a thought-provoking exploration of balance, intelligence, and 
diverse forces shaping our world.  It's not just a logo; it's a visual narrative.

#### Next steps:
None

## 10. YouTube Transcript Summarization

Summarize YouTube video transcripts using the `youtube_transcript_response` function. This is useful for extracting insights from long videos.

In [18]:
summary = youtube_transcript_response("https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s", 
                                      'summarize in bullets and themes', llmgoogle)
print(summary)

Generated YouTube context for query 'https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s': 

Video: https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s
Transcript Summary: content='This is a comprehensive summary of the Google I/O \'25 Keynote based on the provided 
metadata and transcript.\n\n**Key Announcements and Highlights:**\n\n* **Gemini 2.5:**  Google announced 
significant advancements in its Gemini model, boasting improved performance across various benchmarks (LMArena, 
WebDev Arena) and highlighting its capabilities in coding, reasoning, and multi-modal tasks.  Gemini 2.5 Pro and 
Gemini 2.5 Flash were both detailed, with Flash emphasizing speed and efficiency.  A new "Deep Think" mode was also
teased, promising even greater performance but with a focus on further safety evaluations before wider 
release.\n\n* **Gemini App Updates:**  The Gemini app received major updates, including:\n    * **Agent Mode:**  
Allows the app to perform multi-step tasks on behalf of the user (e.g., finding apartments, booking 
appointments).\n    * **Personalized Smart Replies:**  Generates email replies in the user\'s style and tone using 
personal context.\n    * **Gemini Live Enhancements:** Added camera and screen sharing capabilities, along with 
integration with other Google apps.\n\n* **Google Search Enhancements:**\n    * **AI Overviews:**  Continued growth
and expansion, now reaching over 1.5 billion users monthly.\n    * **AI Mode:** A complete reimagining of Google 
Search, leveraging Gemini 2.5 for more complex queries, conversational follow-ups, and dynamic UI adaptation.  
Features like Deep Search (for in-depth research) and personalized suggestions based on user context were 
highlighted.  Agentic capabilities were also integrated, allowing Search to perform tasks like booking tickets and 
making reservations.\n    * **Multimodal Search (Search Live):**  Allows users to use their camera to ask questions
about the real world in real-time.\n\n* **Generative Media Advancements:**\n    * **Lyria 2:**  An improved music 
generation model capable of creating high-fidelity music with vocals.\n    * **Imagen 4:**  A significantly 
improved image generation model with enhanced quality, speed, and text capabilities.\n    * **Veo 3:**  A new video
generation model with Native Audio generation, allowing for the creation of videos with realistic sound effects and
dialogue.\n    * **Flow:** A new AI filmmaking tool combining Veo, Imagen, and Gemini, designed to streamline the 
creative process for filmmakers.\n\n* **Android XR:**  Google showcased its Android XR platform, designed for a 
range of devices including headsets and glasses.  The platform integrates Gemini for enhanced contextual awareness 
and assistance.  Partnerships with Samsung, Gentle Monster, and Warby Parker were announced for headset and glasses
development.  A live demo showcased the capabilities of Android XR glasses.\n\n* **Project Astra and Mariner:**  
These research projects, previously showcased in earlier I/O events, are now being integrated into various Google 
products, demonstrating the transition of research prototypes into real-world applications.\n\n* **New AI 
Subscription Plans:** Google announced two new AI subscription plans: Google AI Pro and Google AI Ultra, offering 
varying levels of access to AI features and capabilities.\n\n**Overall Tone and Message:**\n\nThe keynote 
emphasized Google\'s significant progress in AI, particularly with Gemini, and its commitment to integrating AI 
into its existing products and creating new experiences.  The focus was on making AI more personal, proactive, and 
powerful for users, while also highlighting the potential of AI to accelerate scientific discovery and address 
societal challenges.  The keynote also stressed collaboration with developers and the creative community to shape 
the future of AI.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings':
[]}, 'fin

Video: https://www.youtube.com/watch?v=o8NiE3XMPrM&t=6648s
Transcript Summary: content='This is a comprehensive summary of the Google I/O \'25 Keynote based on the provided 
metadata and transcript.\n\n**Key Announcements and Highlights:**\n\n* **Gemini 2.5:**  Google announced 
significant advancements in its Gemini model, boasting improved performance across various benchmarks (LMArena, 
WebDev Arena) and highlighting its capabilities in coding, reasoning, and multi-modal tasks.  Gemini 2.5 Pro and 
Gemini 2.5 Flash were both detailed, with Flash emphasizing speed and efficiency.  A new "Deep Think" mode was also
teased, promising even greater performance but with a focus on further safety evaluations before wider 
release.\n\n* **Gemini App Updates:**  The Gemini app received major updates, including:\n    * **Agent Mode:**  
Allows the app to perform multi-step tasks on behalf of the user (e.g., finding apartments, booking 
appointments).\n    * **Personalized Smart Replies:**  Generates email replies in the user\'s style and tone using 
personal context.\n    * **Gemini Live Enhancements:** Added camera and screen sharing capabilities, along with 
integration with other Google apps.\n\n* **Google Search Enhancements:**\n    * **AI Overviews:**  Continued growth
and expansion, now reaching over 1.5 billion users monthly.\n    * **AI Mode:** A complete reimagining of Google 
Search, leveraging Gemini 2.5 for more complex queries, conversational follow-ups, and dynamic UI adaptation.  
Features like Deep Search (for in-depth research) and personalized suggestions based on user context were 
highlighted.  Agentic capabilities were also integrated, allowing Search to perform tasks like booking tickets and 
making reservations.\n    * **Multimodal Search (Search Live):**  Allows users to use their camera to ask questions
about the real world in real-time.\n\n* **Generative Media Advancements:**\n    * **Lyria 2:**  An improved music 
generation model capable of creating high-fidelity music with vocals.\n    * **Imagen 4:**  A significantly 
improved image generation model with enhanced quality, speed, and text capabilities.\n    * **Veo 3:**  A new video
generation model with Native Audio generation, allowing for the creation of videos with realistic sound effects and
dialogue.\n    * **Flow:** A new AI filmmaking tool combining Veo, Imagen, and Gemini, designed to streamline the 
creative process for filmmakers.\n\n* **Android XR:**  Google showcased its Android XR platform, designed for a 
range of devices including headsets and glasses.  The platform integrates Gemini for enhanced contextual awareness 
and assistance.  Partnerships with Samsung, Gentle Monster, and Warby Parker were announced for headset and glasses
development.  A live demo showcased the capabilities of Android XR glasses.\n\n* **Project Astra and Mariner:**  
These research projects, previously showcased in earlier I/O events, are now being integrated into various Google 
products, demonstrating the transition of research prototypes into real-world applications.\n\n* **New AI 
Subscription Plans:** Google announced two new AI subscription plans: Google AI Pro and Google AI Ultra, offering 
varying levels of access to AI features and capabilities.\n\n**Overall Tone and Message:**\n\nThe keynote 
emphasized Google\'s significant progress in AI, particularly with Gemini, and its commitment to integrating AI 
into its existing products and creating new experiences.  The focus was on making AI more personal, proactive, and 
powerful for users, while also highlighting the potential of AI to accelerate scientific discovery and address 
societal challenges.  The keynote also stressed collaboration with developers and the creative community to shape 
the future of AI.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings':
[]}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} 
id='run--b7e022

You can also now do simple phrase search and get summarise of topk videos relevant to the search phrase 

In [19]:
summary = youtube_transcript_response("Qwen 3 coder reviews", 
                                      'summarize in bullets and themes', llmgoogle)
print(summary)

[2025-07-27 12:08:47,538] INFO utils.websearch_utils: Found YouTube video: Qwen 3 Coder: NEW Agentic Coding LLM! Insanely Powerful, Fast, & Free! (Opensource) by WorldofAI at https://www.youtube.com/watch?v=8jBw0CVPZPM
[2025-07-27 12:08:47,538] INFO utils.websearch_utils: Found YouTube video: Qwen 3 Coder: NEW Agentic Coding LLM! Insanely Powerful, Fast, & Free! (Opensource) by WorldofAI at https://www.youtube.com/watch?v=8jBw0CVPZPM
[2025-07-27 12:08:55,593] INFO utils.websearch_utils: Found YouTube video: Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutorial by Astro K Joseph at https://www.youtube.com/watch?v=kv57HqEb2j8
[2025-07-27 12:08:55,593] INFO utils.websearch_utils: Found YouTube video: Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutorial by Astro K Joseph at https://www.youtube.com/watch?v=kv57HqEb2j8
[2025-07-27 12:09:04,635] INFO utils.websearch_utils: Found YouTube video: QWEN 3 CODER is Unleashed... better than KIMI K2 by Wes Rot

Generated YouTube context for query 'Qwen 3 coder reviews': 

Video: Qwen 3 Coder: NEW Agentic Coding LLM! Insanely Powerful, Fast, & Free! (Opensource) by WorldofAI
URL: https://www.youtube.com/watch?v=8jBw0CVPZPM
Transcript Summary: content='This is a detailed analysis of the YouTube video metadata and transcript.\n\n**Video 
Overview:**\n\nThe video is a review and demonstration of Qwen3-Coder, a new open-source agentic coding large 
language model (LLM) from Alibaba.  The presenter highlights its capabilities, including its large context window 
(256K tokens, scalable to 1M), its strong performance on coding benchmarks (outperforming some closed-source 
models), and its accompanying command-line interface (CLI), Qwen Code.  The video includes several live demos 
showcasing Qwen3-Coder\'s ability to generate code for various tasks, such as creating animations, interactive 
dashboards, and neural network visualizers.  A key selling point is that the model and its tools are completely 
free and open-source.\n\n**Strengths of the Video:**\n\n* **Clear and Concise Introduction:** The video clearly 
introduces Qwen3-Coder and its key features early on.\n* **Compelling Visuals:** The demos are engaging and 
effectively demonstrate the model\'s capabilities.  The visual results of the code generation are shown, making the
power of the LLM clear to viewers.\n* **Benchmark Comparisons:** The presenter provides benchmark comparisons to 
other LLMs, establishing Qwen3-Coder\'s competitive advantage.\n* **Multiple Access Methods:** The video explains 
multiple ways to access and use Qwen3-Coder (chatbot, API, local installation).\n* **Well-Structured Content:** The
video is logically organized, moving from an introduction to demos and concluding with a call to action.\n* 
**Effective Use of Keywords and Hashtags:** The metadata and description are well-optimized for search, using 
relevant keywords and hashtags.\n* **Sponsor Integration:** The sponsor message is integrated naturally and 
doesn\'t disrupt the flow of the video.\n\n\n**Areas for Improvement:**\n\n* **Transcript Accuracy:** While the 
transcript is generally accurate, there are a few minor typos and inconsistencies (e.g., "Coin 3" instead of "Qwen 
3" in several places).  A thorough proofread would improve the quality.\n* **Demo Complexity:** While the demos are
impressive, some viewers might find them too advanced.  Including simpler demos could broaden the appeal to a wider
audience.\n* **Technical Depth:**  The video could benefit from a slightly deeper dive into the technical aspects 
of Qwen3-Coder\'s architecture (Mixture of Experts) for a more technically inclined audience.  A brief explanation 
of how the 256K context window works would also be beneficial.\n* **Call to Action Clarity:** While the video has 
multiple calls to action (subscribe, donate, join Discord), consolidating them might make them more 
impactful.\n\n\n**Overall:**\n\nThe YouTube video is well-produced and effectively communicates the value 
proposition of Qwen3-Coder.  The engaging demos and clear explanations make it a valuable resource for developers 
and AI enthusiasts interested in open-source coding LLMs.  Addressing the minor areas for improvement would further
enhance its impact and reach.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 
'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} 
id='run--327884b3-8a87-417e-ba81-0e0e748223e1-0' usage_metadata={'input_tokens': 3267, 'output_tokens': 624, 
'total_tokens': 3891, 'input_token_details': {'cache_read': 0}}



Video: Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutorial by Astro K Joseph
URL: https://www.youtube.com/watch?v=kv57HqEb2j8
Transcript Summary: content='Here\'s a breakdown of the YouTube video transcript, organized for clarity and 
SEO:\n\n**Title:** Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutori

Video: Qwen 3 Coder: NEW Agentic Coding LLM! Insanely Powerful, Fast, & Free! (Opensource) by WorldofAI
URL: https://www.youtube.com/watch?v=8jBw0CVPZPM
Transcript Summary: content='This is a detailed analysis of the YouTube video metadata and transcript.\n\n**Video 
Overview:**\n\nThe video is a review and demonstration of Qwen3-Coder, a new open-source agentic coding large 
language model (LLM) from Alibaba.  The presenter highlights its capabilities, including its large context window 
(256K tokens, scalable to 1M), its strong performance on coding benchmarks (outperforming some closed-source 
models), and its accompanying command-line interface (CLI), Qwen Code.  The video includes several live demos 
showcasing Qwen3-Coder\'s ability to generate code for various tasks, such as creating animations, interactive 
dashboards, and neural network visualizers.  A key selling point is that the model and its tools are completely 
free and open-source.\n\n**Strengths of the Video:**\n\n* **Clear and Concise Introduction:** The video clearly 
introduces Qwen3-Coder and its key features early on.\n* **Compelling Visuals:** The demos are engaging and 
effectively demonstrate the model\'s capabilities.  The visual results of the code generation are shown, making the
power of the LLM clear to viewers.\n* **Benchmark Comparisons:** The presenter provides benchmark comparisons to 
other LLMs, establishing Qwen3-Coder\'s competitive advantage.\n* **Multiple Access Methods:** The video explains 
multiple ways to access and use Qwen3-Coder (chatbot, API, local installation).\n* **Well-Structured Content:** The
video is logically organized, moving from an introduction to demos and concluding with a call to action.\n* 
**Effective Use of Keywords and Hashtags:** The metadata and description are well-optimized for search, using 
relevant keywords and hashtags.\n* **Sponsor Integration:** The sponsor message is integrated naturally and 
doesn\'t disrupt the flow of the video.\n\n\n**Areas for Improvement:**\n\n* **Transcript Accuracy:** While the 
transcript is generally accurate, there are a few minor typos and inconsistencies (e.g., "Coin 3" instead of "Qwen 
3" in several places).  A thorough proofread would improve the quality.\n* **Demo Complexity:** While the demos are
impressive, some viewers might find them too advanced.  Including simpler demos could broaden the appeal to a wider
audience.\n* **Technical Depth:**  The video could benefit from a slightly deeper dive into the technical aspects 
of Qwen3-Coder\'s architecture (Mixture of Experts) for a more technically inclined audience.  A brief explanation 
of how the 256K context window works would also be beneficial.\n* **Call to Action Clarity:** While the video has 
multiple calls to action (subscribe, donate, join Discord), consolidating them might make them more 
impactful.\n\n\n**Overall:**\n\nThe YouTube video is well-produced and effectively communicates the value 
proposition of Qwen3-Coder.  The engaging demos and clear explanations make it a valuable resource for developers 
and AI enthusiasts interested in open-source coding LLMs.  Addressing the minor areas for improvement would further
enhance its impact and reach.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 
'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} 
id='run--327884b3-8a87-417e-ba81-0e0e748223e1-0' usage_metadata={'input_tokens': 3267, 'output_tokens': 624, 
'total_tokens': 3891, 'input_token_details': {'cache_read': 0}}



Video: Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutorial by Astro K Joseph
URL: https://www.youtube.com/watch?v=kv57HqEb2j8
Transcript Summary: content='Here\'s a breakdown of the YouTube video transcript, organized for clarity and 
SEO:\n\n**Title:** Qwen Code CLI + Qwen-3 Coder 🔥 | Better Than Claude Code? - Full Tutorial\n\n**Meta 
Description:** Learn how to set up and use the n

# 11. Git Repos Exploration

You can now explore structure and get the content of files/folders in github repositories, you can even plug in local code folders

In [20]:
from utils.git_utils import *

tree = await git_tree_search("https://github.com/SPThole/CoexistAI/")
print(tree)

Directory structure:
└── spthole-coexistai/
    ├── README.md
    ├── __init__.py
    ├── app.py
    ├── coexist_tutorial.ipynb
    ├── LICENSE
    ├── model_config.py
    ├── quick_setup.sh
    ├── README_MCP.md
    ├── requirements.txt
    ├── system_prompt.py
    ├── output/
    │   └── map_with_route_and_pois.html
    ├── searxng/
    │   ├── settings.yml
    │   ├── settings.yml.new
    │   ├── uwsgi.ini
    │   └── uwsgi.ini.new
    └── utils/
        ├── __init__.py
        ├── answer_generation.py
        ├── config.py
        ├── git_utils.py
        ├── map.py
        ├── process_content.py
        ├── reddit_utils.py
        ├── utils.py
        └── websearch_utils.py

In [21]:
#local repo
from utils.git_utils import *

tree = await git_tree_search(".")
print(tree)

Directory structure:
└── CoexistAI/
    ├── README.md
    ├── __init__.py
    ├── app.py
    ├── coexist_tutorial.ipynb
    ├── LICENSE
    ├── lmstudio_chats.ipynb
    ├── model_config.py
    ├── quick_setup.sh
    ├── README_MCP.md
    ├── requirements.txt
    ├── search_context.txt
    ├── system_prompt.py
    ├── output/
    │   └── map_with_route_and_pois.html
    ├── searxng/
    │   ├── settings.yml
    │   ├── settings.yml.new
    │   ├── uwsgi.ini
    │   └── uwsgi.ini.new
    └── utils/
        ├── __init__.py
        ├── answer_generation.py
        ├── config.py
        ├── git_utils.py
        ├── map.py
        ├── process_content.py
        ├── reddit_utils.py
        ├── utils.py
        └── websearch_utils.py

In [22]:
content = await git_specific_content("https://github.com/SPThole/CoexistAI/",
                                  part = "README_MCP.md",
                                  type="file")
print(content)

https://github.com/SPThole/CoexistAI/blob/main//README_MCP.md


================================================
FILE: README_MCP.md
================================================
# CoexistAI v0.0.2 

<p align="center">
  <img src="artifacts/v002mcplogo.jpeg" alt="CoexistAI MCP Logo" width="200"/>
</p>

## 🚀 What's New in v2

- **Direct location search:** You can now search for any place, not just find routes!
- **Advanced Reddit search:** Use your own phrases to search across reddit; results ranked better with BM25 for 
sharper discovery.
- **YouTube power-up:** Search and summarize YouTube using your own search phrases or video URLs and even add a 
prompt for custom responses.
- **Explore to your folders/files**: Explore local folders and files with extended support to diverse files 
including (vision integrated)'.png', '.jpg', '.jpeg', '.gif', '.bmp', '.webp', '.tiff', '.svg', etc, more to come..
- **Sharper web search:** More focused and actionable results than ever before.
- **MCP support everywhere:** Now fully connect coexistai to LM Studio and other MCP hosts—seamless integration! 
[See Guide](README_MCP.md)
- **GitHub & local repo explorer:** Explore ask questions about codebases - works with both GitHub and local repos!

---

## 🛠 Quick Start

### Method (Less flexible but faster):

1. **Clone the repository:**
   ```sh
   git clone https://github.com/SPThole/CoexistAI.git coexistai
   cd coexistai
   ```


2. **Configure your model and embedding settings:**
   
   - Edit `model_config.py` to set your preferred LLM and embedding model.
   - Add LLM and Embedder API Key (for google mode both would be same)
   - Example (for full local mode):
    ```py
    model_config = {
      # Name of the LLM model to use. For local models, use the model name served by your local server.
      "llm_model_name": "google/gemma-3-12b",

      # LLM provider type: choose from 'google', 'local', 'groq', or 'openai' or 'others' 
      # in case of 'others' (base url needs to be updated in openai_compatible given below accordingly).
      # Make sure to update the api_key variable above to match the provider.
      "llm_type": "local",

      # List of tools or plugins to use with the LLM, if any. Set to None if not used.
      "llm_tools": None,

      # Additional keyword arguments for LLM initialization.
      "llm_kwargs": {
        "temperature": 0.1,  # Sampling temperature for generation.
        "max_tokens": None,  # Maximum number of tokens to generate (None for default).
        "timeout": None,     # Timeout for API requests (None for default).
        "max_retries": 2,    # Maximum number of retries for failed requests.
        "api_key": llm_api_key,  # API key for authentication.
      },

      # Name of the embedding model to use.
      # For Google, use their embedding model names. For local/HuggingFace, use the model path or name.
      "embedding_model_name": "nomic-ai/nomic-embed-text-v1",

      "embed_kwargs":{}, #additional kwargs for embedding model initialization

      # Embedding backend: 'google' for Google, 'infinity_emb' for local/HuggingFace models.
      "embed_mode": "infinity_emb",

      # Name of the cross-encoder model for reranking, typically a HuggingFace model.
      "cross_encoder_name": "BAAI/bge-reranker-base"
    }
    ```
   - See the file for all available options and defaults.

3. **Run the setup script:**
   - For macOS or Linux with zsh:
     ```sh
     zsh quick_setup.sh
     ```
   - For Linux with bash:
     ```sh
     bash quick_setup.sh
     ```

   > The script will:
   > - Pull the SearxNG Docker image
   > - Create and activate a Python virtual environment
   > - **USER ACTION NEEDED** Set your `GOOGLE_API_KEY` (edit the script to use your real key). [Obtain your API 
key (Currently Gemini, OpenAI and ollama is supported)](https://ai.google.dev/gemini-api/docs/api-key) from your 
preferred LLM provider. (Only needed when google mode is set, else set in model_config.py)
   > - Start the SearxNG Docker container
   > - Install Python dependenci

## 12. Generating Maps

Generate maps with routes and points of interest using the `generate_map` function. You can visualize directions and locations directly in your notebook.

In [29]:
# Example: Generate a map with a route and POIs
from utils.map import *
s = generate_map("M G Road, Bangalore", "Indiranagar, Bangalore")
from IPython.display import display, HTML
with open("output/map_with_route_and_pois.html") as f:
    html_content = f.read()
# display(HTML(html_content))

[2025-07-27 12:13:04,557] INFO utils.map: Found 3 probable locations for 'M G Road, Bangalore'.
[2025-07-27 12:13:04,558] INFO utils.map: Auto-selected location: Mahatma Gandhi Road, Tasker Town, Shivajinagar, Bengaluru, Bengaluru Central City Corporation, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560001, India
[2025-07-27 12:13:05,549] INFO utils.map: Found 3 probable locations for 'Indiranagar, Bangalore'.
[2025-07-27 12:13:05,551] INFO utils.map: Auto-selected location: Indiranagar, Bengaluru Central City Corporation, Bengaluru, Bangalore East, Bengaluru Urban, Karnataka, 560038, India
[2025-07-27 12:13:05,895] INFO utils.map: Route found between start and end coordinates.
[2025-07-27 12:13:06,491] INFO utils.map: Found 82 POIs near (12.9747828, 77.6096698).
[2025-07-27 12:13:06,874] INFO utils.map: Found 79 POIs near (12.9732913, 77.6404672).
[2025-07-27 12:13:07,000] INFO utils.map: Map generated and saved as 'map_with_route_and_pois.html'.
[2025-07-27 12:13:07,000] 

You can even get the location specific information

In [30]:
# Example: Generate a map with a route and POIs
s = generate_map("M G Road, Bangalore", None, task="location_only")
print(s)
# display(HTML(html_content))

[2025-07-27 12:13:10,669] INFO utils.map: Found 3 probable locations for 'M G Road, Bangalore'.
[2025-07-27 12:13:10,671] INFO utils.map: Auto-selected location: Mahatma Gandhi Road, Tasker Town, Shivajinagar, Bengaluru, Bengaluru Central City Corporation, Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560001, India


Start location: Mahatma Gandhi Road, Tasker Town, Shivajinagar, Bengaluru, Bengaluru Central City Corporation, 
Bengaluru, Bangalore North, Bengaluru Urban, Karnataka, 560001, India (Lat: 12.9747828, Lon: 77.6096698)

## 11. Advanced Query Handling

Handle advanced queries, such as generating detailed reports or toy examples, using `query_web_response`. This enables complex, multi-step reasoning and content generation.

In [23]:
detailed_report = await query_web_response(
    "Give me end to end working for text diffusion model.",
    '26-07-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=1,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-07-27 12:09:49,765] INFO utils.answer_generation: Summary:False,subquery:[''], iscoverd:False
[2025-07-27 12:09:49,765] INFO utils.answer_generation: Summary:False,subquery:[''], iscoverd:False
[2025-07-27 12:09:49,767] INFO utils.websearch_utils: Search response generated for query 'Give me end to end working for text diffusion model.' using pure query.
[2025-07-27 12:09:49,767] INFO utils.websearch_utils: Search response generated for query 'Give me end to end working for text diffusion model.' using pure query.
[2025-07-27 12:09:49,768] INFO utils.websearch_utils: Search phrases for query 'Give me end to end working for text diffusion model.': ['Give me end to end working for text diffusion model.']
[2025-07-27 12:09:49,768] INFO utils.websearch_utils: Search phrases for query 'Give me end to end working for text diffusion model.': ['Give me end to end working for text diffusion model.']
[2025-07-27 12:09:50,420] INFO utils.websearch_utils: Search results for query 'Give me en

#### Answer: 
The provided research paper describes E3 TTS, an end-to-end text-to-speech model using diffusion.  It takes plain 
text as input and generates an audio waveform iteratively.  It doesn't rely on intermediate representations like 
spectrograms or alignment information.  The model's architecture and training process are not fully detailed in the
abstract.  To provide a complete workflow for a general text diffusion model, more information is needed, such as 
the specific model architecture, training data, and hyperparameters.

#### Next steps:
More information is needed to provide a complete workflow.  The provided text focuses on a specific text-to-speech 
model (E3 TTS) and doesn't detail a general text diffusion model workflow.

In [24]:
# You can even summarise full page 
detailed_report = await query_web_response(
    "Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions",
    '27-06-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-07-27 12:10:02,224] INFO utils.answer_generation: Summary:True,subquery:['Summarize Ollama blog post on Secure Minions', 'Extract learning notes from Ollama Secure Minions blog post'], iscoverd:False
[2025-07-27 12:10:02,224] INFO utils.answer_generation: Summary:True,subquery:['Summarize Ollama blog post on Secure Minions', 'Extract learning notes from Ollama Secure Minions blog post'], iscoverd:False
[2025-07-27 12:10:02,225] INFO utils.websearch_utils: Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['summarize ollama blog post on secure minions', 'extract learning notes from ollama secure minions blog post']
[2025-07-27 12:10:02,225] INFO utils.websearch_utils: Search phrases for query 'Summarise this full page in detail, and give me learning notes https://ollama.com/blog/secureminions': ['summarize ollama blog post on secure minions', 'extract learning notes from ollama secure minions blog post'

This blog post from Ollama details Secure Minions, a collaboration with Stanford's Hazy Research lab enhancing the 
open-source Minions project.  Minions allows local LLMs (like Ollama's models) to collaborate with powerful 
cloud-based frontier models (like GPT-4o) efficiently and cost-effectively.  The original Minions protocol, while 
reducing cloud costs significantly (5x-30x) and maintaining high accuracy (98%), still transmitted some potentially
sensitive data to the cloud.

Secure Minions addresses this privacy concern by implementing end-to-end encryption, even from the cloud provider. 
This is achieved using NVIDIA's Hopper H100 GPUs' confidential computing mode.  The result is a system where no 
plaintext is exposed during transmission or remote LLM inference, with minimal latency overhead (less than 1%).

The blog post provides instructions for getting started: cloning the repository, setting up a virtual environment, 
installing the necessary packages (including Ollama and a model like Gemma 3:4b), and running either an interactive
Streamlit app or a Python script demonstrating the secure protocol.  Example code is included to illustrate 
programmatic usage.  The post encourages readers to refer to the HazyResearch blog for complete technical details.


**Learning Notes:**

* **Minions:** A system for efficient and cost-effective collaboration between local and cloud-based LLMs.
* **Secure Minions:** An enhanced version of Minions providing end-to-end encryption for enhanced privacy.
* **Confidential Computing:** Leveraging hardware capabilities (NVIDIA Hopper H100 GPUs) to protect data during 
processing and transmission.
* **Practical Implementation:**  The blog post provides clear instructions and example code for using Secure 
Minions.
* **Key Improvement:**  Secure Minions significantly improves the privacy of local-first LLM setups by preventing 
sensitive data from leaving the device.
* **Minimal Overhead:** The added security comes with minimal performance impact (less than 1% latency increase).


**Assumptions Made:**

* The reader has basic familiarity with command-line interfaces, Git, Python, and virtual environments.
* The reader has access to a system with sufficient resources to run the required software and models (including 
potentially an NVIDIA Hopper H100 GPU for optimal performance, although this is not explicitly stated as a 
requirement).
* The reader understands the basic concepts of LLMs and their applications.

In [25]:
# or multiple pages
detailed_report = await query_web_response(
    "Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp",
    '27-06-2025',
    'Saturday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=[],
    local_mode=False,
    split=False
)
print(detailed_report[0])

[2025-07-27 12:10:19,356] INFO utils.answer_generation: Summary:True,subquery:['Summarize SecureMinions blog', 'Summarize llama.cpp github'], iscoverd:True
[2025-07-27 12:10:19,356] INFO utils.answer_generation: Summary:True,subquery:['Summarize SecureMinions blog', 'Summarize llama.cpp github'], iscoverd:True
[2025-07-27 12:10:19,358] INFO utils.websearch_utils: Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['summarize secureminions blog', 'summarize llama.cpp github']
[2025-07-27 12:10:19,358] INFO utils.websearch_utils: Search phrases for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['summarize secureminions blog', 'summarize llama.cpp github']
[2025-07-27 12:10:19,358] INFO utils.websearch_utils: Extracted URLs from query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp': ['https://ollama.com/blog/secureminions', 'ht

Summarising using 2 documents

[2025-07-27 12:10:30,668] INFO utils.websearch_utils: Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.
[2025-07-27 12:10:30,668] INFO utils.websearch_utils: Response generated for query 'Summarise https://ollama.com/blog/secureminions and https://github.com/ggml-org/llama.cpp'.


Here's a summary addressing the user's query, combining information from the provided documents:

**Summary:**

The Ollama blog post details "Secure Minions," a collaboration between Ollama and Stanford's Hazy Research lab.  
Secure Minions enhances the original Minions project, which allows local LLMs (like Ollama's models) to collaborate
with powerful cloud-based frontier models (like GPT-4o) while minimizing data transfer to the cloud.  The key 
improvement in Secure Minions is the implementation of end-to-end encryption using NVIDIA's Hopper H100 GPUs' 
confidential computing mode. This ensures that no plaintext data is exposed during transmission or inference, even 
from the cloud provider.  The overhead is minimal (less than 1% added latency), even with large models and long 
prompts.  The blog post provides instructions for setting up and running Secure Minions, including using a provided
Streamlit app and example Python code.

The GitHub repository for `llama.cpp` describes a C/C++ library for LLM inference.  It aims for minimal setup and 
high performance across various hardware platforms.  It supports several backends (including Metal, CUDA, HIP, and 
others), allowing for inference on CPUs, GPUs, and other accelerators.  The repository includes tools like 
`llama-cli` (a command-line interface), `llama-server` (an OpenAI API-compatible server), and `llama-bench` (for 
benchmarking).  It also provides instructions for obtaining and quantizing models, primarily using the GGUF format,
and offers support for models from Hugging Face and other sources.


**Assumptions Made:**

* No assumptions were made beyond the information explicitly presented in the provided documents.  The summary 
directly reflects the content of the blog post and the GitHub repository description.

In [26]:
# you can even do search/summary within local files

# or multiple pages
detailed_report = await query_web_response(
    "Read the code file and for each explain me what it does in concise manner, 2 lines max for each",
    '27-06-2025',
    'Friday',
    searcher,
    hf_embeddings,
    True,
    cross_encoder,
    llmgoogle,
    text_model=llmgoogle,
    num_results=3,
    document_paths=['utils/map.py','app.py'],
    local_mode=True,
    split=False
)
print(detailed_report[0])

[2025-07-27 12:10:35,780] INFO utils.answer_generation: Summary:True,subquery:['Explain each code file concisely', 'Give 2 lines max for each explanation'], iscoverd:False
[2025-07-27 12:10:35,780] INFO utils.answer_generation: Summary:True,subquery:['Explain each code file concisely', 'Give 2 lines max for each explanation'], iscoverd:False
[2025-07-27 12:10:35,781] INFO utils.websearch_utils: Search phrases for query 'Read the code file and for each explain me what it does in concise manner, 2 lines max for each': ['explain each code file concisely', 'give 2 lines max for each explanation']
[2025-07-27 12:10:35,781] INFO utils.websearch_utils: Search phrases for query 'Read the code file and for each explain me what it does in concise manner, 2 lines max for each': ['explain each code file concisely', 'give 2 lines max for each explanation']
[2025-07-27 12:10:35,782] WARNING utils.websearch_utils: Please add list of paths as input, earlier it used to be list of list
[2025-07-27 12:10

Summarising using 13 documents

[2025-07-27 12:10:46,010] INFO utils.websearch_utils: Response generated for query 'Read the code file and for each explain me what it does in concise manner, 2 lines max for each'.
[2025-07-27 12:10:46,010] INFO utils.websearch_utils: Response generated for query 'Read the code file and for each explain me what it does in concise manner, 2 lines max for each'.


The code implements a FastAPI application providing various search functionalities.  It uses several utility 
modules (`utils/map.py`, `utils/websearch_utils.py`, `utils/reddit_utils.py`, `utils/git_utils.py`) for web search,
Reddit search, map-based route planning with POIs, and Git repository interaction.  The application also includes 
functionality for summarizing web pages and YouTube videos, and checking the completeness of research.  A large 
language model (LLM) is used for several tasks, including response generation and summarization.  The 
`model_config.py` file is assumed to contain configurations for the LLM and embedding models.  The application uses
a SearxNG docker container for web searches (assumed to be started automatically if not already running).  The 
`utils/map.py` module uses the Overpass and Nominatim APIs for POI retrieval and geocoding, and the OSRM API for 
route calculation.  The map is generated using Folium and saved as an HTML file.

The next cell demonstrates how to summarize Reddit posts and comments from the "OpenAI" subreddit using the `reddit_reader_response` function. It retrieves the top 2 "hot" posts and summarizes their content with the help of the `llmgoogle` generative model. This is useful for quickly extracting key insights from Reddit discussions.

In [27]:
from utils.reddit_utils import *
summary = reddit_reader_response(
  subreddit="OpenAI",
  url_type="hot",
  n=2,
  k=2,
  custom_url=None,
  time_filter="today",
  search_query=None,
  sort_type="hot",
  model=llmgoogle
)
print(summary)

Fetching posts from /r/OpenAI

Fetching comments for post: Agent casually clicking the "I am not a robot" button

Fetching comments for post: Offering researchers $1 billion is not normal

This Reddit summary covers two separate posts from r/OpenAI.  Neither post contains substantial discussion; the 
comments are brief and offer limited elaboration on the post titles.

The first post, titled "Agent casually clicking the 'I am not a robot' button" 
(https://www.reddit.com/r/OpenAI/comments/1m9c15h/agent_casually_clicking_the_i_am_not_a_robot/),  received a score
of 2502. The post itself contains no text. The comments are limited to: "That’s hilarious" and  ""This step is 
necessary to prove I'm not a bot"".  These comments are humorous observations on the irony of an AI agent 
performing a CAPTCHA designed to prevent automated actions.

The second post, "Offering researchers $1 billion is not normal" 
(https://www.reddit.com/r/OpenAI/comments/1m9pgjo/offering_researchers_1_billion_is_not_normal/), scored 1180.  
This post also lacks substantive text. The comments speculate on the motivations behind such large offers. One 
comment expresses skepticism about Mark Zuckerberg's judgment, citing his metaverse investments as an example of 
poor decision-making.  Another comment suggests that mega-wealthy individuals might believe they are the only ones 
capable of guiding AI development towards "recursive self-improvement," implying a belief in the necessity of their
involvement in directing AI's future.  Neither comment provides concrete evidence to support their claims.

In [28]:
from utils.reddit_utils import *
summary = reddit_reader_response(
  subreddit="",
  url_type="search",
  n=2,
  k=2,
  custom_url=None,
  time_filter="today",
  search_query="Qwen 3 coder reviews",
  sort_type="hot",
  model=llmgoogle
)
print(summary)

Fetching posts from search search

Fetching comments for post: Is there a way to use qwen 3 coder inside vs code or cursor

Fetching comments for post: Kimi K2 vs Qwen 3 Coder - Coding Tests

This Reddit summary focuses on user reviews of the Qwen 3 Coder large language model (LLM), specifically concerning
its use in coding contexts within VS Code and other IDEs.

**Reddit Post 1:  
(https://www.reddit.com/r/LocalLLaMA/comments/1m79sp9/is_there_a_way_to_use_qwen_3_coder_inside_vs_code/)**

This post asks about integrating Qwen 3 Coder into VS Code or Cursor.  The comments suggest using extensions that 
support OpenAI-compatible APIs.  Specifically, Roocode for VS Code is mentioned as an option that can connect to 
local models or OpenRouter.

**Reddit Post 2: (https://www.reddit.com/r/ChatGPTCoding/comments/1m7myxo/kimi_k2_vs_qwen_3_coder_coding_tests/)**

This post compares Qwen 3 Coder to Kimi K2 in coding tasks across various IDEs (VSCode, Cline, Roo Code, Windsurf).
The author conducted tests and provides the following observations:

* **Overall Performance:**  Kimi K2 performed better overall in the author's tests.  The author notes that Kimi K2 
felt comparable to Sonnet 4 in many aspects.

* **Tool Calling:** Qwen 3 Coder exhibited near-flawless tool calling within VSCode, while Kimi K2 struggled with 
this functionality, similar to Gemini 2.5 Pro.  However, Qwen 3 Coder also performed well with Roo Code's tool 
calls.

* **Instruction Following:** Kimi K2 demonstrated superior instruction following compared to Qwen 3 Coder.

* **Cost:** Qwen 3 Coder proved significantly more expensive, particularly when using Alibaba as the inference 
provider (up to $60 per million tokens).  Switching to other providers within OpenRouter reduced the cost. Kimi K2 
was approximately half the price of Qwen 3 Coder.

* **Safety:**  The author warns about potential safety issues with Kimi K2 in Windsurf, noting that it deleted a 
development database in Azure after receiving a dangerous command.  The author recommends denying dangerous 
commands in Windsurf's settings.

* **Video:** A video demonstrating one of the tests is linked: (https://youtu.be/ljCO7RyqCMY).  This video likely 
provides further visual detail on the comparative performance of the two models.


In summary, while Qwen 3 Coder shows promise in its tool-calling capabilities within certain IDEs, Kimi K2 appears 
to outperform it in overall performance and instruction following, at a significantly lower cost.  However, safety 
concerns regarding dangerous command execution were raised for Kimi K2.  The cost of using Qwen 3 Coder is heavily 
dependent on the chosen inference provider.

---

This concludes the tutorial for the **coexist**. You have learned how to perform web search, document processing, answer generation, YouTube summarization, and more. For further details, refer to the project README or explore the codebase for advanced usage patterns. 